### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip uninstall --yes tensorboard-plugin-wit
%cd ..

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/to-run-for-results/nsl=kdd/baselines
Obtaining file:///project/to-run-for-results/nsl%3Dkdd/baselines
  Attempting uninstall: baselines
    Found existing installation: baselines 0.1.6
    Uninstalling baselines-0.1.6:
      Successfully uninstalled baselines-0.1.6
  Running setup.py develop for baselines
Requirement already up-to-date: tensorflow==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
Requirement already up-to-date: tensorflow-gpu==1.14.0 in /opt/conda/lib/python3.7/site-packages (1.14.0)
/project/to-run-for-results/nsl=kdd


In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime

from stable_baselines.deepq.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import deepq
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17345289795390436920,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9198654532665190881
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8995719860617919838
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
nslkdd_train = pd.read_feather("/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTrain.feather")
nslkdd_test = pd.read_feather("/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTest.feather")

nslkdd_train = pd.get_dummies(nslkdd_train,columns=['protocol_type' ,'service', 'flag'])
nslkdd_test = pd.get_dummies(nslkdd_test, columns=['protocol_type' ,'service', 'flag'])
nslkdd_train.head(5)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
nslkdd_train.info()
nslkdd_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Columns: 123 entries, duration to flag_SH
dtypes: float64(15), int64(23), object(1), uint8(84)
memory usage: 47.6+ MB


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
count,125973.00000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.00000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,0.395736,0.279250,...,0.08917,0.012399,0.000818,0.019218,0.276655,0.002897,0.001008,0.000389,0.594929,0.002151
std,2604.51531,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,0.489010,23.942042,...,0.28499,0.110661,0.028583,0.137292,0.447346,0.053750,0.031736,0.019719,0.490908,0.046332
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.00000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,42908.00000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,1.000000,7479.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [5]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [6]:
def nslkdd_split_df(train_df, test_df):
    train_col = train_df.columns[-1]
    print(train_col)
    train_cols = train_df.columns[:-1]
    print(train_cols)
    test_col = test_df.columns[-1]
    print(test_col)
    test_cols = test_df.columns[:-1]
    print(test_cols)

    return train_df[train_cols], train_df[train_col], test_df[test_cols], test_df[test_col]

In [7]:
print(nslkdd_train['class'].value_counts())
nslkdd_train['class'] = nslkdd_train['class'].astype('object')
atk_idx = nslkdd_train.loc[nslkdd_train['class'] != "normal"].index
nslkdd_train.loc[atk_idx, 'class'] = 1.0
nslkdd_train.loc[nslkdd_train.index.difference(atk_idx), 'class'] = 0.0
nslkdd_train['class'] = nslkdd_train['class'].astype(dtype=np.float32)

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: class, dtype: int64


In [8]:
print(nslkdd_test['class'].value_counts())
nslkdd_test['class'] = nslkdd_test['class'].astype('object')
atk_idx = nslkdd_test.loc[nslkdd_test['class'] != "normal"].index
nslkdd_test.loc[atk_idx, 'class'] = 1.0
nslkdd_test.loc[nslkdd_test.index.difference(atk_idx), 'class'] = 0.0
nslkdd_test['class'] = nslkdd_test['class'].astype(dtype=np.float32)

normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
loadmodule            2
phf                   2
sqlattack             2
perl                  2
worm                  2
udpstorm              2
imap                  1
Name: class, dtype: int64


In [9]:
extra_removables = nslkdd_test.columns ^ nslkdd_train.columns
print(extra_removables)
try:
    nslkdd_train = nslkdd_train.drop(labels=extra_removables, axis='columns')
except:
    pass
try:
    nslkdd_test.drop(labels=extra_removables, axis='columns')
except:
    pass
nslkdd_train.info()

Index(['service_aol', 'service_harvest', 'service_http_2784',
       'service_http_8001', 'service_red_i', 'service_urh_i'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Columns: 117 entries, duration to flag_SH
dtypes: float32(1), float64(15), int64(23), uint8(78)
memory usage: 46.4 MB


In [10]:
x_train, y_train, x_test, y_test  = nslkdd_split_df(nslkdd_train, nslkdd_test)

flag_SH
Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       ...
       'flag_OTH', 'flag_REJ', 'flag_RSTO', 'flag_RSTOS0', 'flag_RSTR',
       'flag_S0', 'flag_S1', 'flag_S2', 'flag_S3', 'flag_SF'],
      dtype='object', length=116)
flag_SH
Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       ...
       'flag_OTH', 'flag_REJ', 'flag_RSTO', 'flag_RSTOS0', 'flag_RSTR',
       'flag_S0', 'flag_S1', 'flag_S2', 'flag_S3', 'flag_SF'],
      dtype='object', length=116)


In [11]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()

# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


### Custom environment

In [12]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        # change to 19 if detectiong all different attacks
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(nslkdd_train.columns) - 1,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [13]:
def ids_ddqn():
    
    env = IdsEnv(images_per_episode=1)

    model = deepq.DQN(
        MlpPolicy, 
        env, 
        policy_kwargs=dict(dueling=False),
        double_q=False,
        verbose=1, 
        learning_rate=0.00025,
        buffer_size=1000000,
        exploration_fraction=0.1,
        exploration_final_eps=0.1,
        train_freq=4,
        learning_starts=50000,
        target_network_update_freq=10000,
        gamma=1.0,
        prioritized_replay=False,
        prioritized_replay_alpha=0.6,
        batch_size=32,
    )
    model.learn(
        total_timesteps=int(1.0e5),
        log_interval=int(1.0e4),
    )

    env.close()
    
    return model

start_time = time.time()
ddqn_model_1 = ids_ddqn()
print()
print("DQN 1 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_2 = ids_ddqn()
print()
print("DQN 2 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_3 = ids_ddqn()
print()
print("DQN 3 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_4 = ids_ddqn()
print()
print("DQN 4 Training Time:", time.time() - start_time)


/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


--------------------------------------
| % time spent exploring  | 10       |
| episodes                | 10000    |
| mean 100 episode reward | 0.1      |
| steps                   | 9999     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 20000    |
| mean 100 episode reward | 0.1      |
| steps                   | 19999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 30000    |
| mean 100 episode reward | 0.1      |
| steps                   | 29999    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 9        |
| episodes                | 40000    |
| mean 100 episode reward | 0.1      |
| steps                   | 39999    |
--------------------------------------
--------------------------------------
| % time spent exploring 

In [14]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        try:
            recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        except:
            recall = 0
        try:
            FPR = (float(FP) / (TN + FP)) # 1 - specificity
        except:
            FPR = 0
        try:
            f1_score = 2 * (precision * recall) / (precision + recall)
        except:
            f1_score = 0
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]

models = [ddqn_model_1, ddqn_model_2, ddqn_model_3, ddqn_model_4]

results_1 = ids_eval(ddqn_model_1)
results_2 = ids_eval(ddqn_model_2)
results_3 = ids_eval(ddqn_model_3)
results_4 = ids_eval(ddqn_model_4)
total_results = [-1,-1,-1,-1,-1]
accuracies = [results_1[0], results_2[0], results_3[0], results_4[0]]

for i in range(len(results_1)):
    total_results[i] = (results_1[i] + results_2[i] + results_3[i] + results_4[i] )/ 4

print()    
print('Total validation done...')
print('Accuracy: {0}%'.format(total_results[0] * 100))
print('Precision: {0}%'.format(total_results[1] * 100))
print('Recall/TPR/Sensitivity: {0}%'.format(total_results[2] * 100))
print('FPR: {0}%'.format(total_results[3] * 100))
print('F1 score: {0}'.format(total_results[4]))

highest = 0
for i in range(4):
    if accuracies[i] > accuracies[highest]:
        highest = i

print()
print("Saving model {0}".format(highest + 1))
models[highest].save('vanilla_dqn_cicddos2019_allfeat_2.pkl')




validation done...
Accuracy: 53.64209032029101%
Precision: 53.58860195903829%
Recall/TPR/Sensitivity: 99.78444702371083%
FPR: 99.46564885496183%
F1 score: 0.6972944788830311

validation done...
Accuracy: 91.77535267500664%
Precision: 92.11041852181656%
Recall/TPR/Sensitivity: 99.60519980741454%
FPR: 100.0%
F1 score: 0.9571131158917419

validation done...
Accuracy: 82.00692041522491%
Precision: 82.03918076580588%
Recall/TPR/Sensitivity: 99.88074588031223%
FPR: 98.53444064484611%
F1 score: 0.9008506893517161

validation done...
Accuracy: 82.02466506964777%
Precision: 82.32413178984862%
Recall/TPR/Sensitivity: 99.55847512384234%
FPR: 100.0%
F1 score: 0.9012478065899786

Total validation done...
Accuracy: 77.36225712004259%
Precision: 77.51558325912734%
Recall/TPR/Sensitivity: 99.70721695881998%
FPR: 99.500022374952%
F1 score: 0.8641265226791168

Saving model 2
